In [1]:
import re

In [2]:
import requests

In [3]:
STORAGE_PATH = "/media/decarv/Seagate Portable Drive/data"

In [138]:
len(filenames)

50

In [3]:
import os
import re
import requests
from urllib.parse import urljoin
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from PyPDF2 import PdfMerger, PdfReader
from pdfminer.high_level import extract_text

class WebCrawler:
    def __init__(self, base_url):
        self.base_url = base_url
        self.session = requests.Session()

    def get_html(self, url):
        resp = self.session.get(url)
        if resp.ok:  # Raise exception if status is not 200
            return resp.text
        else:
            return None

    def get_links(self, html, selector):
        soup = BeautifulSoup(html, 'html.parser')
        elements = soup.select(selector)
        return [el.get('href') for el in elements]

    def get_pdf_links(self, start=1, end=100):
        pdf_dict = {}
        for i in range(start, end):
            url = f'{self.base_url}{i}'
            html = self.get_html(url)
            if html is None:
                continue
            links = self.get_links(html, 'div.dadosDocNome a')
            for link in links:
                _id = re.findall('tde-\d+-\d+', link)
                if _id:
                    _id = _id[0]
                    html = self.get_html(link)
                    pdf_links = self.get_links(html, 'a[href$=".pdf"]')
                    pdf_dict[_id] = [urljoin(link, pdf_link) for pdf_link in pdf_links]
        return pdf_dict

    def download_pdf_and_extract_text(self, pdf_dict):
        merger = PdfMerger()
        for _id, pdf_links in pdf_dict.items():
            for pdf_link in pdf_links:
                file_name = os.path.join(STORAGE_PATH, f'theses/{_id}.pdf')
                urlretrieve(pdf_link, file_name)
                merger.append(PdfReader(file_name, 'rb'))
                text = extract_text(file_name)
                with open(os.path.join(STORAGE_PATH, f'text/{_id}.txt'), 'w') as f:
                    f.write(text)
        merger.write("combined.pdf")
        merger.close()
        
    def clean_and_send_to_weaviate(self, pdf_dict):
        # Connect to Weaviate
        password = getpass("Please enter your Weaviate password: ")
        client = Client("http://localhost:8080", AuthClientPassword("username", password))

        for _id, _ in pdf_dict.items():
            os.path.join(STORAGE_PATH, f'text/{_id}.txt')

            # Read the file and remove extra whitespaces
            with open(file_name, 'r') as f:
                text = f.read()
                cleaned_text = re.sub('\s+', ' ', text).strip()

            # Write the cleaned text back into the file
            with open(file_name, 'w') as f:
                f.write(cleaned_text)

            # Send the cleaned text to Weaviate
            client.data_object.create({
                "class": "Document",
                "id": _id,
                "properties": {
                    "text": cleaned_text,
                },
            }, "replace")

In [4]:
crawler = WebCrawler('https://www.teses.usp.br/index.php?option=com_jumi&fileid=19&Itemid=87&lang=en&g=4&b7=Medicine&c7=c&o7=AND&pagina=')

In [16]:
pdf_dict = crawler.get_pdf_links(1,101)

In [17]:
pdf_dict

{'tde-02062005-101952': ['http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Capa.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Conclusion.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Introduct.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/MatMet.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Referencias.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Result1.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Result10.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Result2.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Result3.pdf',
  'http://www.teses.usp.br/teses/disponiveis/59/59135/tde-02062005-101952/publico/Result4.pdf',
  'http://w

In [42]:
crawler.download_pdf_and_extract_text(pdf_dict)

Xref table not zero-indexed. ID numbers for objects will be corrected.
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-02062005-101952.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
Xref table not zero-indexed. ID numbers for objects will be corrected.
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-02062005-101952.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
Xref table not zero-indexed. ID numbers for objects will be corrected.
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-02062005-101952.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this

The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-01042014-114150.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-13042023-091810.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-11062013-085036.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-15012018-074054.pdf'> contains a m

The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-30032023-135455.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-03012022-153229.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-22092010-140235.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/media/decarv/Seagate Portable Drive/data/theses/tde-23092015-142247.pdf'> contains a m

HTTPError: HTTP Error 404: Not Found

In [5]:
import json

from weaviate import Client

In [7]:
client = Client("http://localhost:8080")

In [130]:
schema = {
  "classes": [
    {
      "class": "theses4",
      "properties": [
        {
          "name": "text",
          "dataType": ["string"]
        },
        {
          "name": "reference",
          "dataType": ["string"]
        }
      ]
    }
  ]
}

client.schema.create(schema)

In [85]:
def clean_text(pdf_dict):
    for _id, _ in pdf_dict.items():
        file_name = os.path.join(STORAGE_PATH, f'text/{_id}.txt')
        try:
            # Read the file and remove extra whitespaces
            with open(file_name, 'r') as f:
                text = f.read()
                cleaned_text = re.sub('\s+', ' ', text).strip()

            # Write the cleaned text back into the file
            with open(file_name, 'w') as f:
                f.write(cleaned_text)
        except Exception:
            pass

<>:8: DeprecationWarning: invalid escape sequence '\s'
<>:8: DeprecationWarning: invalid escape sequence '\s'
/tmp/ipykernel_5643/256304299.py:8: DeprecationWarning: invalid escape sequence '\s'
  cleaned_text = re.sub('\s+', ' ', text).strip()


In [86]:
from getpass import getpass
from weaviate import Client, AuthClientPassword
import asyncio
from aiofile import AIOFile
import re

In [131]:
class WeaviateUploader:
    def __init__(self):
        self.client = Client("http://localhost:8080")

    def send_to_weaviate(self, filename):
        reference = filename.split('.')[0].split("/")[-1]

        # Read the file and remove extra whitespaces
        with open(filename, 'r') as fp:
            text = fp.read()
            
        i = text.find("RESUMO")

        # Create a data object in Weaviate
        data = {
            "class": "theses4",
            "reference": reference,
            "text": text[i:3000+i],
        }
        self.client.data_object.create(data, "theses3")

In [132]:
uploader = WeaviateUploader()

In [133]:
for filename in filenames:
    uploader.send_to_weaviate(filename)
    print(f"SENT: {filename}")

{'class': 'theses4', 'reference': 'tde-14012013-115721', 'text': 'RESUMO Anguera, MG. Exposição à radiação não-ionizante emitida por equipamentos terapêuticos de micro-ondas e morbidade referida em fisioterapeutas [tese doutorado]. São Paulo: Faculdade de Medicina, Universidade de São Paulo; 2012. 115 pp. Introdução: O fisioterapeuta utiliza-se de vários equipamentos eletro- eletrônicos que emitem radiação eletromagnética para fins terapêuticos. Entre esses, o equipamento de diatermia por micro-ondas de alta frequência, sendo essa de 2,45 GHz. Nesse procedimento terapêutico, é possível ter perdas variáveis da energia irradiada por reflexão e por dispersão, desde o aplicador até a área a ser tratada. Assim, o profissional que opera o equipamento de diatermia por micro-ondas, é exposto a essa radiação diariamente durante anos. Adicionalmente a esse cenário ocupacional, observa-se que o fisioterapeuta, de modo geral, tem raras atitudes de proteção ocupacional durante essas aplicações, tor

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-15012018-074054.txt
{'class': 'theses4', 'reference': 'tde-15052013-193530', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-15052013-193530.txt
{'class': 'theses4', 'reference': 'tde-16112017-085817', 'text': 'RESUMO Alves SB. Como os acordos de parceria podem melhorar o desempenho das atividades de PD&I na área de fármacos e medicamentos [Tese]. São Paulo: Faculdade de Medicina, Universidade de São Paulo; 2017. Ambientes altamente dinâmicos, complexos e que envolvam conhecimento, alta tecnologia, e recursos diversos, estimulam acordos de parcerias, que passam a ser um meio de transferência de conhecimento e tecnologia, de acesso a ativos complementares e de busca da inovação. Neste trabalho, coloca-se a questão de como os acordos de parcerias e sua gestão, mais especificamente, os acordos das Parcerias de Desenvolvimento Produtivo (PDPs), podem melhorar o desempenho das atividades de Pesquisa, Desenvolvimen

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-21062010-175305.txt
{'class': 'theses4', 'reference': 'tde-22072013-144625', 'text': 'RESUMO iii ANTONIASSI, M. O espalhamento Compton de raios X no diagnóstico de neoplasias mamárias. [tese]. Ribeirão Preto: Faculdade de Filosofia Ciências e Letras de Ribeirão Preto, Universidade de São Paulo; 2012. 102p. Neste trabalho foram realizadas medidas de espalhamento elástico e inelástico em tecidos mamários normais e neoplásicos (benignos e malignos) usando uma energia de 17,44 keV (radiação Kα – Mo) e um ângulo de espalhamento de 90º (x = 0,99Å-1) com o objetivo de determinar as densidades eletrônicas (ρe), número atômico efetivo (Zef) e largura a meia altura (LMA) do pico de espalhamento inelástico destes tecidos. Também foram estudadas por meio de simulações computacionais imagens obtidas por técnica de tomografia de espalhamento Compton. As metodologias experimentais foram verificadas através da análise de materiais de referência.

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-23092015-142247.txt
{'class': 'theses4', 'reference': 'tde-24042017-213315', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-24042017-213315.txt
{'class': 'theses4', 'reference': 'tde-24072020-151438', 'text': 'RESUMO SUMMARY 1. Introdução. ..........................................................................................................................1 1.1 O tratamento da FA através da ablação com cateter por radiofrequência e o esôfago......... 1 1.2 A relação anatômica entre o esôfago e o átrio esquerdo. ................................................... 2 1.3 Fatores de risco para formação da fístula atrioesofágica .................................................... 6 1.4 Fisiopatologia da lesão térmica esofágica ......................................................................... 6 1.5 Apresentação clínica ...................................................................................

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-25102017-114401.txt
{'class': 'theses4', 'reference': 'tde-26042016-083838', 'text': 'RESUMO ABSTRACT 1 INTRODUÇÃO ............................................................................................. 2 1.1 Papel do ecocardiograma transesofágica bidimensional na oclusão percutânea da CIA .......................................................................... 6 1.2 Evolução da ecocardiografia transesofágica tridimensional em tempo real e seu papel na oclusão da CIA ............................................... 11 2 OBJETIVOS .............................................................................................. 16 2.1 Objetivo Primário .................................................................................. 16 2.2 Objetivo secundário ............................................................................. 16 3 MATERIAL E MÉTODO .....................................................................

...

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-29092016-142720.txt
{'class': 'theses4', 'reference': 'tde-30032023-135455', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-30032023-135455.txt
{'class': 'theses4', 'reference': 'tde-30112016-142327', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-30112016-142327.txt
{'class': 'theses4', 'reference': 'tde-31102018-102140', 'text': 'RESUMO ABSTRACT INTRODUÇÃO ............................................................................................... 1 1.1 1.2 1.3 1.4 METODOLOGIA ............................................................................................ Por que história? ......................................................................................... O recorte do tempo (1936-1970) ................................................................ Recortes no espaço e a história institucional .............................................. Fontes ...............

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-02012018-112131.txt
{'class': 'theses4', 'reference': 'tde-02062005-101952', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-02062005-101952.txt
{'class': 'theses4', 'reference': 'tde-02122014-164155', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-02122014-164155.txt
{'class': 'theses4', 'reference': 'tde-03012022-153229', 'text': ''}
SENT: /media/decarv/Seagate Portable Drive/data/text/tde-03012022-153229.txt
{'class': 'theses4', 'reference': 'tde-03092008-091923', 'text': 'RESUMO ............................................................................................................ xv ABSTRACT .......................................................................................................... 1 Capítulo 1 .............................................................................................................. 1 1. Introdução ............................................

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-07112016-095116.txt
{'class': 'theses4', 'reference': 'tde-08062015-083115', 'text': 'RESUMO SUMMARY 1 INTRODUÇÃO...........................................................................................23 1.2 Importância da hipertensão arterial sistêmica.....................................24 1.3 Sistema nervoso simpático e hipertensão arterial sistêmica..............26 1.3.1 Via eferente..............................................................................26 1.3.2 Via aferente..............................................................................27 1.4 Denervação simpática.........................................................................30 1.5 Estudos clínicos de denervação simpática renal................................31 1.5.1 Symplicity HTN-1......................................................................31 1.5.2 Symplicity HTN-2....................................................................

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-09022021-094039.txt
{'class': 'theses4', 'reference': 'tde-09062021-140814', 'text': 'RESUMO Andrade, R. T. S. A experiência dos usuários com o diagnóstico e o tratamento da infecção por HIV em dois serviços do SUS da região metropolitana do estado da Bahia [Tese]. São Paulo: Faculdade de Medicina, Universidade de São Paulo, 2020. A boa qualidade da assistência prestada pelos serviços de saúde para pessoas que vivem com HIV está relacionada aos resultados clínicos individuais do tratamento e ao impacto na transmissão do HIV na sociedade. A qualidade é definida por três atributos principais: efetividade, segurança e centralidade nas necessidades e preferências do paciente. A experiência do usuário dos serviços de saúde é um dos principais componentes do atributo de centralidade. O programa brasileiro de HIV/Aids conta com ferramentas que monitoram os resultados clínicos do tratamento. Tem também estimulado inquéritos sobre a dimen

SENT: /media/decarv/Seagate Portable Drive/data/text/tde-11062013-085036.txt
{'class': 'theses4', 'reference': 'tde-11082021-145720', 'text': 'RESUMO Araujo WRM. Prevalência e fatores associados ao atraso no desenvolvimento neuropsicomotor em crianças de uma coorte na região Amazônica [tese]. São Paulo: Faculdade de Medicina, Universidade de São Paulo; 2020. Condições adversas durante o período pré-natal, ao nascimento e na primeira infância podem levar a déficits no aprendizado, comportamento e na saúde física e mental ao longo da vida. Os estudos de coorte de nascimento se destacam nesse cenário por permitir a análise dos efeitos de exposições precoces sobre a saúde e desenvolvimento ao longo da vida. Os objetivos da presente tese foram (i) realizar uma síntese de evidências sobre as principais características dos estudos de coorte iniciados no período pré-natal e no nascimento no mundo e no Brasil, e (ii) determinar a frequência e os fatores associados ao atraso no desenvolvimento n

### Queries

#### Get com `nearText`

In [137]:
import weaviate
import json

# client definido
nearText = {"concepts": ["ressonância"]}
result = (
    client.query
    .get("theses3", ["reference", "text"])
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

result

{'data': {'Get': {'Theses3': [{'reference': 'tde-24042017-213315',
     'text': 'DE DE SÃO PAULO Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto Departamento de Física Jeam Haroldo Oliveira Barbosa Conteúdo de íons paramagnéticos no cérebro e sua correlação com mapas quantitativos de Ressonância Magnética Ribeirão Preto – SP 2017 I I O L V E R A B A R B O S A J E A M H A R O L D O R e s s o n â n c i a M a g n é t i c a s u a c o r r e l a ç ã o c o m m a p a s q u a n t i t a t i v o s d e C o n t e ú d o d e í o n s p a r a m a g n é t i c o s n o c é r e b r o e DOUTORADO FFCLRP/USP 2017 JEAM HAROLDO OLIVEIRA BARBOSA Conteúdo de íons paramagnéticos no cérebro e sua correlação com mapas quantitativos de Ressonância Magnética Tese apresentada à Faculdade de Filosofia Ciências e Letras de Ribeirão Preto da Universidade de São Paulo para obtenção do título de Doutor em Ciências Área de Concentração: Física aplicada à Medicina e Biologia. Orientador: Carlos Ernesto Garrido Sa

In [100]:
nearText = {"concepts": ["ressonância"]}
result = (
    client.query
    .get("theses3", ["reference", "text"])
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

result

{'data': {'Get': {'Theses3': [{'reference': 'tde-24042017-213315',
     'text': 'DE DE SÃO PAULO Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto Departamento de Física Jeam Haroldo Oliveira Barbosa Conteúdo de íons paramagnéticos no cérebro e sua correlação com mapas quantitativos de Ressonância Magnética Ribeirão Preto – SP 2017 I I O L V E R A B A R B O S A J E A M H A R O L D O R e s s o n â n c i a M a g n é t i c a s u a c o r r e l a ç ã o c o m m a p a s q u a n t i t a t i v o s d e C o n t e ú d o d e í o n s p a r a m a g n é t i c o s n o c é r e b r o e DOUTORADO FFCLRP/USP 2017 JEAM HAROLDO OLIVEIRA BARBOSA Conteúdo de íons paramagnéticos no cérebro e sua correlação com mapas quantitativos de Ressonância Magnética Tese apresentada à Faculdade de Filosofia Ciências e Letras de Ribeirão Preto da Universidade de São Paulo para obtenção do título de Doutor em Ciências Área de Concentração: Física aplicada à Medicina e Biologia. Orientador: Carlos Ernesto Garrido Sa

#### TODO

- Text to Vec: como funciona;
- Experimentos: tempo para execucao da base de dados / tempo de insercao / tempo de query;
- Descobrir uma métrica para testar a acurácia;
- Tentar entender o que eu to fazendo, a partir das métricas (pode ser alto nível, sem entrar muito em detalhes).

### Extração dos Resumos

In [49]:
class Thesis:

    def __init__(self, **kwargs):
        self.doi = kwargs.get("doi", None)
        self.type = kwargs.get("type", None)
        self.author = kwargs.get("author", None)
        self.institute = kwargs.get("institute", None)
        self.knowledge_area = kwargs.get("knowledge_area", None)
        self.committee = kwargs.get("committee", None)
        self.title_pt = kwargs.get("title_pt", None)
        self.title_en = kwargs.get("title_en", None)
        self.keywords_pt = kwargs.get("keywords_pt", None)
        self.keywords_en = kwargs.get("keywords_en", None)
        self.abstract_pt = kwargs.get("abstract_pt", None)
        self.abstract_en = kwargs.get("abstract_en", None)
        self.publish_date = kwargs.get("publish_date", None)

    def __repr__(self):
        return str(self.__dict__)

    def parse_webpage(self, webpage):
        import re
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(webpage, 'html.parser')

        raw_metadata = soup.find_all(class_="DocumentoTexto")
        raw_metadata_keys = soup.find_all(class_="DocumentoTituloTexto")
        metadata = {
            k.text.strip().lower(): re.sub(r"\s+", " ", v.text.strip()) 
            for (k, v) in zip(raw_metadata_keys, raw_metadata)
        }
        self.doi = metadata["doi"]
        self.type = metadata["documento"]
        self.author = metadata["autor"]
        self.institute = metadata["unidade da usp"]
        self.knowledge_area = metadata["área do conhecimento"]
        self.committee = metadata["banca examinadora"]
        self.title_pt = metadata["título em português"]
        self.keywords_pt = metadata["palavras-chave em português"]
        self.title_en = metadata["título em inglês"]
        self.keywords_en = metadata["palavras-chave em inglês"]
        self.publish_date = metadata["data de publicação"]

        raw_data = soup.find_all(class_="DocumentoTextoResumo")
        raw_data_keys = soup.find_all(class_="DocumentoTituloTexto2")
        data = {
            k.text.strip().lower(): re.sub(r"\s+", " ", v.text.strip()) 
            for (k, v) in zip(raw_data_keys, raw_data)
        }
        self.abstract_pt = data["resumo em português"]
        self.abstract_en = data["resumo em inglês"]


In [55]:
import os
import re
import requests
from urllib.parse import urljoin
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from PyPDF2 import PdfMerger, PdfReader
from pdfminer.high_level import extract_text


class Crawler:
    """
    This class structures all the theses into Theses objects from a search url.
    """
    
    def __init__(self):
        self.base_urls = {}
        self.theses = {}
        self.errors = []
        self.session = requests.Session()

    
    def crawl(self):
        for base_url, v in self.base_urls.items():
            urls = self.extract_webpages_urls(base_url)
            for url in urls:
                self.parse_thesis(url)

    def add_base_url(self, url):
        if type(url) is list:
            add_base_url(url.pop())
            add_base_url(url)
        else:
            if url not in self.base_urls:
                self.base_urls[url] = False
            else:
                raise Exception("Base url already added.")
        
    def extract_webpages_urls(self, start: int = 1, end: int = 2) -> None:
        for i in range(start, end):
            url = f'{self.base_url}{i}'
            search_webpage = requests.get(url, headers={"Accept-Language": "pt-br,pt-BR"})
            if search_webpage is None or not search_webpage.ok:
                self.errors.append(url)
                continue
        
        soup = BeautifulSoup(search_webpage.content, 'html.parser')
        elements = soup.select("div.dadosDocNome a")
        return [el.get('href') for el in elements]
        
    def parse_thesis(self, url):
        webpage = requests.get(url, headers={"Accept-Language": "pt-br,pt-BR"})
        if not webpage.ok:
            continue
        thesis = Thesis()
        thesis.parse_webpage(webpage.content)
        [id] = re.findall('tde-\d+-\d+', url)
        self.theses[id] = thesis

SyntaxError: 'continue' not properly in loop (353320935.py, line 54)

In [53]:
crawler = Crawler()

TypeError: Crawler.__init__() missing 1 required positional argument: 'base_url'

In [52]:
crawler.theses

{'tde-02062005-101952': {'doi': '10.11606/T.59.2005.tde-02062005-101952', 'type': 'Tese de Doutorado', 'author': 'Aggarwal, Lucimara Perpétua Ferreira (Catálogo USP)', 'institute': 'Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto', 'knowledge_area': 'Física Aplicada à Medicina e Biologia', 'committee': 'Borissevitch, Iouri (Presidente) Baptista, Mauricio da Silva Ito, Amando Siuiti Louro, Sonia Renaux Wanderley Tabak, Marcel', 'title_pt': 'Interações das porfirinas aquo-solúveis TPPS4 e TMPyP com sistemas biológicos e modelos. Efeitos do pH e da força iônica.', 'title_en': 'Interaction of water-soluble porphyrins TPPS4 and TMPyP with biological and model systems. Effects pf pH and ionic strength.', 'keywords_pt': 'agregação cultura celular neoplásica porfirinas sistemas biomiméticos', 'keywords_en': 'aggregation biomimetic systems neoplasic cells culture porphyrins', 'abstract_pt': 'As porfirinas e seus derivados têm sido, ao longo dos anos, alvos de vários campos diferente